In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from csv import writer
from urllib.request import Request,urlopen
# !pip install requests-html
# from requests_html import HTMLSession,AsyncHTMLSession
# !pip3 install nest_asyncio
# import nest_asyncio
# nest_asyncio.apply()
# !wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | sudo apt-key add -
# !echo 'deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main' | sudo tee /etc/apt/sources.list.d/google-chrome.list
# !sudo apt update 
# !sudo apt install google-chrome-stable
# !ldd ~/.local/share/pyppeteer/local-chromium/588429/chrome-linux/chrome | grep 'not found'


In [2]:
from http.client import HTTPException
from pyppeteer.errors import BrowserError
import pyppeteer.launcher
import time
from urllib.error import URLError
from urllib.request import urlopen

def get_ws_endpoint(url) -> str:
    url = url + '/json/version'
    timeout = time.time() + 30
    while True:
        if time.time() > timeout:
            raise BrowserError('Browser closed unexpectedly:\n')
        try:
            with urlopen(url) as f:
                data = json.loads(f.read().decode())
            break
        except (URLError, HTTPException) as e:
            pass
        time.sleep(0.1)

    return data['webSocketDebuggerUrl']


pyppeteer.launcher.get_ws_endpoint = get_ws_endpoint

ModuleNotFoundError: ignored

In [ ]:
url="https://www.practo.com/search/doctors?results_type=doctor&q=%5B%7B%22word%22%3A%22Dentist%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22subspeciality%22%7D%5D&city=Delhi"

asession=AsyncHTMLSession()

async def afunc():
  
  page=await asession.get(url)
  await page.html.arender(timeout=50)
  return page
page=asession.run(afunc)
# lists=[]

# soup=BeautifulSoup(page.text,"html.parser")
# lists=soup.find_all('div',class_="info-section")
# for i in lists:
  
#   print(i.find('a')['href'])
# next=soup.find('div',class_="pure-u-14-24")
# .find('ul',class_="c-paginator")

# select_one("a:nth-of-type(4)").text

# print(next)

In [ ]:
url="https://www.practo.com/delhi/doctor/dr-anurag-ahuja-dentist-1/info?practice_id=1141246&specialization=Dentist&referrer=doctor_listing"
url="https://www.practo.com//delhi/doctor/shreya-krishna-dentist?practice_id=1353928&specialization=Dentist&referrer=doctor_listing"
page=requests.get(url)
lists=[]

soup=BeautifulSoup(page.content,"html.parser")
name=soup.find('h1', class_="c-profile__title").text
qual=soup.find('p', class_="c-profile__details").text
spl=soup.find('div', class_="u-d-inline-flex").text

exp=soup.find('div',class_="c-profile__details")
# print(exp.select_one("h2:nth-of-type(4)"))
exp=exp.select_one("h2:nth-of-type(4)")


upvote_perc=soup.find('span', class_="u-green-text")
fee=soup.find('span', class_="u-f-right u-large-font u-bold u-valign--middle u-lheight-normal").text
# print(name,qual,spl,exp,upvote_perc,fee)

lists=soup.find_all('div', class_="c-profile--clinic--item")
location=[]

timing=[]
temp=[]
loctim=[]

for i in lists:
  if i.find('div', class_="c-profile--clinic--item")!=None:
    temp.append(i.find('div', class_="c-profile--clinic--item"))
for i in temp:
  location.append(i.find('h4',class_="c-profile--clinic__location").text)
  days=i.find_all('p', class_="timings__days")
  time=i.find_all('p', class_="timings__time")
  # print(days)
  # print(time)
  if (len(days)>0 and len(time)>0):
    for j, k in zip(days,time):
      timing.append([j.text,k.text])
    loctim.append([i.find('h4',class_="c-profile--clinic__location").text,[j.text,k.text]])
print(loctim)
  # timing.append(i.find('div', class_= "pure-u-1-3 u-cushion--left").text)
  
revlists=soup.find_all('div',class_="pure-u-22-24 u-lheight-default")
reviews=[]
for i in revlists:
  Dict={}
  typeofc=i.find('span',class_="u-blue-2-background u-smallest-font u-color--white u-round-corner--12 u-cushion--right-less u-cushion--small-left u-cushion--small-top u-cushion--small-bottom").text
  titleofc=i.find('span',class_="procedure").text
  # print(titleofc)
  temp=i.find_all('span',class_="feedback__context")
  keywords=[]
  for j in temp:
    keywords.append(j.text)
  review=i.find('p',class_="feedback__content u-large-font u-d-inline").text
  voteofc=i.find('div',class_="u-cushion--small-bottom u-large-font").text
  Dict['typeofc']=typeofc
  Dict['titleofc']=titleofc
  Dict['voteofc']=voteofc
  Dict['keywords']=keywords
  Dict['review']=review
  reviews.append(Dict)
with open('doctor.csv', 'w', encoding='utf8', newline='') as f:
    wrt = writer(f)
    header = ['Name', 'Qualification', 'Speciality', 'Experience',"Fee","Upvote Percentage","Location and Timings","Type of Consultation","Topic Of Consultation","Vote","Keywords","Review"]
    wrt.writerow(header)
    
    for i in reviews:
      # print()
      info=[name,qual,spl,exp,fee,upvote_perc,loctim,i['typeofc'],i['titleofc'],i['voteofc'],i['keywords'],i['review']]
      wrt.writerow(info)


In [ ]:
from bs4 import BeautifulSoup
import re
from csv import writer
# from urllib.request import Request,urlopen
from requests_html import HTMLSession,AsyncHTMLSession

def get_details(doctor_page):
    lists=[]
    soup=BeautifulSoup(doctor_page.text,"html.parser")
    name=soup.find('h1', class_="c-profile__title")
    qual=soup.find('p', class_="c-profile__details")
    spl=soup.find('div', class_="u-d-inline-flex")

    exp=soup.find('div',class_="c-profile__details")

    if(exp!=None):
        # exp=exp.select_one("h2:nth-of-type(3)")
        # print(exp.text)
        exp = re.findall(r'\d+', str(exp.text))[0]
        # print(exp)

    upvote_perc=soup.find('span', class_="u-green-text")
    fee=soup.find('span', class_="u-f-right u-large-font u-bold u-valign--middle u-lheight-normal")
    # # print(name,qual,spl,exp,upvote_perc,fee)
    #
    lists=soup.find_all('div', class_="c-profile--clinic--item")
    location=[]

    timing=[]
    temp=[]
    loctim=[]

    for i in lists:
        if i.find('div', class_="c-profile--clinic--item")!=None:
            temp.append(i.find('div', class_="c-profile--clinic--item"))
    for i in temp:
        location.append(i.find('h4',class_="c-profile--clinic__location").text)
        days=i.find_all('p', class_="timings__days")
        time=i.find_all('p', class_="timings__time")

        if len(days)>0 and len(time)>0:
            for j, k in zip(days,time):
                timing.append([j.text,k.text])
            loctim.append([i.find('h4',class_="c-profile--clinic__location").text,[j.text,k.text]])
    # print(loctim)
    # timing.append(i.find('div', class_= "pure-u-1-3 u-cushion--left").text)

    revlists=soup.find_all('div',class_="pure-u-22-24 u-lheight-default")
    reviews=[]
    for i in revlists:
        Dict={}
        typeofc=i.find('span',class_="u-blue-2-background u-smallest-font u-color--white u-round-corner--12 u-cushion--right-less u-cushion--small-left u-cushion--small-top u-cushion--small-bottom")
        titleofc=i.find('span',class_="procedure")
        # print(titleofc)
        temp=i.find_all('span',class_="feedback__context")
        keywords=[]
        for j in temp:
            keywords.append(j.text)
        review=i.find('p',class_="feedback__content u-large-font u-d-inline")
        voteofc=i.find('div',class_="u-cushion--small-bottom u-large-font")

        Dict['typeofc']=typeofc
        Dict['titleofc']=titleofc
        Dict['voteofc']=voteofc
        Dict['keywords']=keywords
        Dict['review']=review
        reviews.append(Dict)
    # print(name)
    rows=[]
    for i in reviews:

        row=[name,qual,spl,fee,upvote_perc,i['typeofc'],i['titleofc'],i['voteofc'],i['review']]
        for j in range(len(row)):
            if row[j]!=None:
                row[j]=row[j].text
        row.insert(3,exp)
        row.insert(6,loctim)
        row.insert(10,i['keywords'])
        rows.append(row)
    return rows
    # print([name,qual,spl,exp,fee,upvote_perc,loctim,reviews])
    # return [name,qual,spl,exp,fee,upvote_perc,loctim,reviews]


def write(rows):
    with open('doctor.csv', 'a', encoding='utf8', newline='') as f:
        wrt = writer(f)
        # header = ['Name', 'Qualification', 'Speciality', 'Experience',"Fee","Upvote Percentage","Location and Timings","Type of Consultation","Topic Of Consultation","Vote","Keywords","Review"]
        # wrt.writerow(header)

        for i in rows:
            # print()
            info=[]
            for j in i:
                info.append(j)
            # info=[name,qual,spl,exp,fee,upvote_perc,loctim,i['typeofc'],i['titleofc'],i['voteofc'],i['keywords'],i['review']]
            wrt.writerow(info)


with open('doctor.csv', 'w', encoding='utf8', newline='') as f:
    wrt = writer(f)
    header = ['Name', 'Qualification', 'Speciality', 'Experience',"Fee","Upvote Percentage","Location and Timings","Type of Consultation","Topic Of Consultation","Vote","Keywords","Review"]
    wrt.writerow(header)


urls=["https://www.practo.com/search/doctors?results_type=doctor&q=%5B%7B%22word%22%3A%22Dentist%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22subspeciality%22%7D%5D&city=Delhi",
     "https://www.practo.com/search/doctors?results_type=doctor&q=%5B%7B%22word%22%3A%22Gynecologist%2Fobstetrician%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22subspeciality%22%7D%5D&city=Delhi",
     "https://www.practo.com/search/doctors?results_type=doctor&q=%5B%7B%22word%22%3A%22Dermatologist%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22subspeciality%22%7D%5D&city=Delhi",
     "https://www.practo.com/search/doctors?results_type=doctor&q=%5B%7B%22word%22%3A%22Ear-nose-throat%20(ent)%20Specialist%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22subspeciality%22%7D%5D&city=Delhi",
     "https://www.practo.com/search/doctors?results_type=doctor&q=%5B%7B%22word%22%3A%22General%20Physician%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22subspeciality%22%7D%5D&city=Delhi",
     "https://www.practo.com/search/doctors?results_type=doctor&q=%5B%7B%22word%22%3A%22orthopedist%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22subspeciality%22%7D%5D&city=Delhi",
     "https://www.practo.com/search/doctors?results_type=doctor&q=%5B%7B%22word%22%3A%22pediatrician%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22subspeciality%22%7D%5D&city=Delhi",
     "https://www.practo.com/search/doctors?results_type=doctor&q=%5B%7B%22word%22%3A%22ophthalmologist%2F%20eye%20surgeon%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22subspeciality%22%7D%5D&city=Delhi",
     "https://www.practo.com/search/doctors?results_type=doctor&q=%5B%7B%22word%22%3A%22cardiologist%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22subspeciality%22%7D%5D&city=Delhi",
     "https://www.practo.com/search/doctors?results_type=doctor&q=%5B%7B%22word%22%3A%22gastroenterologist%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22subspeciality%22%7D%5D&city=Delhi",
     "https://www.practo.com/search/doctors?results_type=doctor&q=%5B%7B%22word%22%3A%22nephrologist%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22subspeciality%22%7D%5D&city=Delhi",
     "https://www.practo.com/search/doctors?results_type=doctor&q=%5B%7B%22word%22%3A%22neurologist%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22subspeciality%22%7D%5D&city=Delhi",
     "https://www.practo.com/search/doctors?results_type=doctor&q=%5B%7B%22word%22%3A%22oncologist%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22subspeciality%22%7D%5D&city=Delhi",
     "https://www.practo.com/search/doctors?results_type=doctor&q=%5B%7B%22word%22%3A%22plastic%20surgeon%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22subspeciality%22%7D%5D&city=Delhi",
     "https://www.practo.com/search/doctors?results_type=doctor&q=%5B%7B%22word%22%3A%22psychiatrist%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22subspeciality%22%7D%5D&city=Delhi",
     "https://www.practo.com/search/doctors?results_type=doctor&q=%5B%7B%22word%22%3A%22pulmonologist%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22subspeciality%22%7D%5D&city=Delhi",
     "https://www.practo.com/search/doctors?results_type=doctor&q=%5B%7B%22word%22%3A%22urologist%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22subspeciality%22%7D%5D&city=Delhi",
     "https://www.practo.com/search/doctors?results_type=doctor&q=%5B%7B%22word%22%3A%22Homoeopath%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22subspeciality%22%7D%5D&city=Delhi"]
count=[200,200,100,100,100,100,100,95,68,51,33,38,58,36,61,45,44,66]

session=HTMLSession()
for l in range(len(urls)):
    url=urls[l]
    for k in range(int(count[l]/10)):

        page=session.get(url)
        j=True
        while(j):
            try:
                page.html.render(timeout=40)
                j=False
            except:
                j=True

        soup=BeautifulSoup(page.text,"html.parser")
        lists=soup.find_all('div',class_="info-section")
        for i in lists:
            link="https://www.practo.com/"+i.find('a')['href']
            print(link)
            doctor_page=session.get(link)
            # print(doctor_page)
            rows=get_details(doctor_page)
            write(rows)


        print(page)
        try:
            pageno=page.html.find('ul.c-paginator',first=True).find('li')
            next_url=list(pageno[11].absolute_links)[0]
            url=next_url
        except:

            print(k,url)
            continue